## Pipeline Acceidents_v2

Using Seaborn or Plotly to make better plots

For making maps I used this: https://jcutrer.com/python/learn-geopandas-plotting-usmaps

In [1]:
import pandas as pd
import numpy as np

#import wget

#plotty things
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode
import plotly.express as px

import pyproj

#mappy things
import geopandas as gpd
import shapely.geometry # to make the interactive plotly pipe shapes work 
from shapely.geometry import Point  #To display lat-long locations on the maps


pd.options.display.max_columns = 999


## Getting the shape data 
https://www.eia.gov/maps/layer_info-m.php for the shape files

'''
#### download a zipped shapefile
wget.download("https://plotly.github.io/datasets/ne_50m_rivers_lake_centerlines.zip")

### open a zipped shapefile with the zip:// pseudo-protocol
geo_df = gpd.read_file("zip://ne_50m_rivers_lake_centerlines.zip")

'''

In [2]:
states = gpd.read_file('data/mapping/usa-states-census-2014.shp') #reading in the USA shape file 

# #using plotly to plot the USA states based on region
# fig = px.choropleth(states,
#                     locations='STUSPS', 
#                     locationmode="USA-states", 
#                     scope="usa",
#                     color='region',
#                     color_continuous_scale="Viridis_r", 
                    
#                     )
# fig.show()

In [3]:
## Reading in the HGL (Hydrocarbon Liquid Gass) transmission pieline data
HGL_trans_pipe = gpd.read_file('data/mapping/HGL_Pipelines_US_202001.shp')

 #using the geopandas clip() function, and clipping the pipeline data to the continental USA boundaries.
HGL_trans_pipe_clipped = gpd.clip(HGL_trans_pipe, states) 

#Gulf of Mexico Pipelines (undersea)
#https://www.data.boem.gov/Main/Mapping.aspx
Gulf_mexico_pipe = gpd.read_file('data/mapping/ppl_arcs.shp')

#interstate and intrastate pipes
inter_intra_pipe = gpd.read_file('data/mapping/NaturalGas_Pipelines_US_202001.shp')

In [ ]:
HGL_trans_pipe.sample(2)

 Moral of the story, you have to give BOTH your data frame and the lat/long/name intot he line_geo, and then use the hover data and true flase dictionaries turn on or turn off what you like. 

In [7]:
#geo_df_trans = pd.read_pickle('geo_df_trans.pkl')

##uncomment if you dont hav pikle file

# # Preparing the geo_dataframe we need from extracting point data fro the shape file. 

# lats = [] #latitude array
# lons = [] #longitude array
# names = [] #pipeline name
# opnames = [] #pipeline operator name

# #for each row of the geopandas get the linestrings, name and oprnames and then unpack the linestrings/multilinestrings into latlong points and add in the names, 
# for feature, name, opname in zip(HGL_trans_pipe_clipped.geometry, HGL_trans_pipe_clipped.Pipename, HGL_trans_pipe_clipped.Opername):
#     if isinstance(feature, shapely.geometry.linestring.LineString):
#         linestrings = [feature]
#     elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
#         linestrings = feature.geoms
#     else:
#         continue
#     for linestring in linestrings: #unpack the linestrings into lat/long
#         x, y = linestring.xy
#         lats = np.append(lats, y)
#         lons = np.append(lons, x)
#         names = np.append(names, [name]*len(y))
#         opnames = np.append(opnames, [opname]*len(y))

#         #need a NaN to stop plotly attaching the end of the pipelines together with arbitrary lines.
#         lats = np.append(lats, None)
#         lons = np.append(lons, None)
#         names = np.append(names, None)
#         opnames = np.append(opnames, None)

# #putting the pipelines on the usa map, with pipeline name as the hover data
# #You now need to make a df of the np arrays to give you flexibility with plotly (especially hover data manipulation)
# numpy_array = np.vstack((lats, lons, names,opnames)) #vertically staking the np arrays to make data frames

# geo_df_trans = pd.DataFrame(np.transpose(numpy_array), 
#              columns=['lats', 
#                       'lons', 'Pipe_Name','Operator_Name'])

# geo_df_trans['Category'] = 'Major Gas Transmission Pipeline'
# geo_df_trans.to_pickle('geo_df_trans.pkl')

# geo_df_trans.sample(3)

,lats,lons,Pipe_Name,Operator_Name,Category
179,41.566257,-88.211819,North System,ONEOK,Major Gas Transmission Pipeline
518,40.533461,-81.100497,Utopia East,KINDER MORGAN,Major Gas Transmission Pipeline
1311,40.403082,-81.106865,Utica East Ohio Midstream,WILLIAMS,Major Gas Transmission Pipeline


In [ ]:
#plotting the shape file onto the USA map using plotly
fig = px.line_geo(geo_df_trans,lat = geo_df_trans.lats, lon = geo_df_trans.lons, hover_name = geo_df_trans.Pipe_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form pipe
                             'lons':False, #remove longitude form pipe
                             'Operator_Name': True, 
                               })

fig.update_traces(line_color='#000000', line_width=1) #updating the default lines used for HGL piepes


fig.update_layout(title="Major Hydrocarbon Gas Liquid Transmission Pipelines")
#fig.update_layout(hovermode="closest")

fig.update_geos(
    resolution=50,
    showland=True, landcolor="#b1f699",
    showocean=False, oceancolor="LightBlue",
    showlakes=True, lakecolor="LightBlue",
    showrivers=False, rivercolor="LightBlue")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



In [11]:
geo_df = pd.read_pickle('geo_df.pkl')

##uncomment if you dont hav pikle file

# # plotting inter_intra pipeline onto the USA map using plotly. 

# lats = [] #latitude array
# lons = [] #longitude array
# opnames = [] #pipeline operator name

# #for each row of the geopandas get the linestrings, name and oprnames and then unpack the linestrings/multilinestrings into latlong points and add in the names, 
# for feature, opname in zip(inter_intra_pipe.geometry, inter_intra_pipe.Operator):
#     if isinstance(feature, shapely.geometry.linestring.LineString):
#         linestrings = [feature]
#     elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
#         linestrings = feature.geoms
#     else:
#         continue
#     for linestring in linestrings: #unpack the linestrings into lat/long
#         x, y = linestring.xy
#         lats = np.append(lats, y)
#         lons = np.append(lons, x)
#         opnames = np.append(opnames, [opname]*len(y))

#         #need a NaN to stop plotly attaching the end of the pipelines together with arbitrary lines.
#         lats = np.append(lats, None)
#         lons = np.append(lons, None)
#         opnames = np.append(opnames, None)

# #putting the pipelines on the usa map, with pipeline name as the hover data
# #You now need to make a df of the np arrays to give you flexibility with plotly (especially hover data manipulation)
# numpy_array = np.vstack((lats, lons, opnames)) #vertically staking the np arrays to make data frames

# geo_df = pd.DataFrame(np.transpose(numpy_array), 
#              columns=['lats', 
#                       'lons','Operator_Name'])

# geo_df['Category'] = 'Inter/Intra State Distribution Pipeline'  
# 
# geo_df.to_pickle('geo_df.pkl')                    


In [ ]:

#finally plotting the dataframe we made with lats, longs and names etc form the HGL shape file, we manipulate the hoverdata to display what we like

fig = px.line_geo(geo_df,lat = geo_df.lats, lon = geo_df.lons, hover_name = geo_df.Operator_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form pipe
                             'lons':False, #remove longitude form pipe
                               })

fig.update_traces(line_color='#BCBCBC', line_width=0.5) #updating the default lines used for HGL piepes


fig.update_layout(title="Inter/Intra State Gas Pipelines")
#fig.update_layout(hovermode="closest")

fig.update_geos(
    resolution=50,
    showland=True, landcolor="#b1f699",
    showocean=False, oceancolor="LightBlue",
    showlakes=True, lakecolor="LightBlue",
    showrivers=False, rivercolor="LightBlue")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [4]:
#geo_df_mx = pd.read_pickle('geo_df_mx.pkl')

##uncomment if you dont hav pikle file

# # plotting Gulf of Mexico pipeline onto the USA map using plotly. 

lats = [] #latitude array
lons = [] #longitude array
opnames = [] #pipeline operator name

#for each row of the geopandas get the linestrings, name and oprnames and then unpack the linestrings/multilinestrings into latlong points and add in the names, 
for feature, opname in zip(Gulf_mexico_pipe.geometry, Gulf_mexico_pipe.SDE_COMPAN):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings: #unpack the linestrings into lat/long
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        opnames = np.append(opnames, [opname]*len(y))

        #need a NaN to stop plotly attaching the end of the pipelines together with arbitrary lines.
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        opnames = np.append(opnames, None)

#putting the pipelines on the usa map, with pipeline name as the hover data
#You now need to make a df of the np arrays to give you flexibility with plotly (especially hover data manipulation)
numpy_array = np.vstack((lats, lons, opnames)) #vertically staking the np arrays to make data frames

geo_df_mx = pd.DataFrame(np.transpose(numpy_array), 
             columns=['lats', 
                      'lons','Operator_Name'])

geo_df_mx['Category'] = 'Gulf of Mexico Pipeline'

geo_df_mx.to_pickle('geo_df_mx.pkl')

In [ ]:

#finally plotting the dataframe we made with lats, longs and names etc form the HGL shape file, we manipulate the hoverdata to display what we like

fig = px.line_geo(geo_df_mx,lat = geo_df_mx.lats, lon = geo_df_mx.lons, hover_name = geo_df_mx.Operator_Name, locationmode="USA-states", scope="usa",
                 hover_data={'lats':False, #remove latitude form pipe
                             'lons':False, #remove longitude form pipe
                               })

fig.update_traces(line_color='#8723e3', line_width=0.25) #updating the default lines used for HGL piepes


fig.update_layout(title="Gulf of Mexico Gas Pipelines")
#fig.update_layout(hovermode="closest")

fig.update_geos(
    resolution=50,
    showland=True, landcolor="#b1f699",
    showocean=False, oceancolor="LightBlue",
    showlakes=True, lakecolor="LightGrey",
    showrivers=False, rivercolor="LightBlue")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



In [ ]:
#Putting it all together

states_map = states.plot(cmap='YlGn', figsize=(18, 81))
#crude_pipe_clipped.plot(ax = states_map, color = 'black')
HGL_trans_pipe_clipped.plot(ax =states_map, color = 'black', linewidth = 1)
Gulf_mexico_pipe.plot(ax =states_map, color = 'purple', linewidth = 0.5)
# #inter_intra_pipe_clipped = geopandas.clip(inter_intra_pipe, states)  #using the clip() function, and clipping the pipeline data to the continental USA boundaries.
inter_intra_pipe.plot(ax = states_map, color = 'orange', linewidth = 0.5)

#setting the plot limits instead of geopandas clipping as it is a lot faster

plt.xlim(-130,-63)
plt.ylim(23,50)

plt.axis('off')

# Next Step: Reading in the incident data

In [2]:
HGL_trans_accidents = pd.read_csv('data/incident_gas_transmission_gathering_jan2010_present.csv',low_memory=False) #changed encoding by saving as csv file again, tab delimited

HGL_trans_accidents.shape

(1521, 625)

The data file we have has so many columns that are not useful, so I have made a .txt file of the columns required for our analysis.

In [3]:
req_clmns = []

with open('data/Gas_Trans_required_columns.txt') as f: #importing the list of required columns form the text file
    lines = f.readlines()
f.close()

for l in lines:
    req_clmns.append(l.strip()) #getting rid of the "\n" escape character



In [4]:
HGL_trans_acc = HGL_trans_accidents[req_clmns]
# I have discovered some outliers that fall outside the continental USA, so I will explude those: 
HGL_trans_acc = HGL_trans_acc[(-140<HGL_trans_acc['LOCATION_LONGITUDE']) & (HGL_trans_acc['LOCATION_LONGITUDE']<-50)]
HGL_trans_acc = HGL_trans_acc[HGL_trans_acc['LOCATION_LATITUDE']<50]

HGL_trans_acc.shape

(1512, 61)

In [ ]:
HGL_trans_acc.sample(5)

In [ ]:
#Putting it all together


states_map = states.plot(cmap='summer', figsize=(81, 60))
# #crude_pipe_clipped.plot(ax = states_map, color = 'black')
# HGL_trans_pipe_clipped.plot(ax = states_map, color = 'blue')
# Gulf_mexico_pipe.plot(ax = states_map, color = 'indigo')

# #inter_intra_pipe_clipped = geopandas.clip(inter_intra_pipe, states)  #using the clip() function, and clipping the pipeline data to the continental USA boundaries.
# #inter_intra_pipe_clipped.plot(ax = states_map, color = 'dimgrey')
# inter_intra_pipe.plot(ax = states_map, color = 'dimgrey')



# Creating lists for lat-long from the dataset
long = HGL_trans_acc['LOCATION_LONGITUDE'].tolist()
lat = HGL_trans_acc['LOCATION_LATITUDE'].tolist()

#geenrating points for the map
geometry = [Point(xy) for xy in zip(long,lat)]

#Putting the generated points in a geopandas dataframe
geo_df = gpd.GeoDataFrame(geometry = geometry)

#Putting the dots on the map
g = geo_df.plot(ax = states_map, markersize = 40, color = 'red',marker = '*')

#setting the plot limits instead of geopandas clipping as it is a lot faster
plt.xlim(-130,-63)
plt.ylim(23,50)



plt.show()

# Getting a feel for the incident data using plotly

In [5]:
HGL_trans_acc.head()

,REPORT_RECEIVED_DATE,IYEAR,REPORT_NUMBER,NAME,OPERATOR_STATE_ABBREVIATION,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,ACCOMPANYING_LIQUID,CAUSE,CAUSE_DETAILS,FATALITY_IND,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,FATAL,INJURY_IND,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,INJURE,GAS_CONSUMED_BY_FIRE_IN_MCF,EXPLODE_IND,INCIDENT_AREA_TYPE,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,ITEM_INVOLVED,PIPE_TYPE,PIPE_DIAMETER,PIPE_WALL_THICKNESS,PIPE_SMYS,PIPE_SPECIFICATION,PIPE_SEAM_TYPE,PIPE_MANUFACTURER,PIPE_COATING_TYPE,PIPE_COATING_DETAILS,COATING_APPLIED_IND,MATERIAL_INVOLVED,RELEASE_TYPE,LEAK_TYPE,RELEASE_TYPE_DETAILS,EST_COST_OPER_PAID,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,EST_COST_OTHER_DETAILS,GAS_COST_IN_MCF,NUM_PERSONS_HOSP_NOT_OVNGHT,NUM_INJURED_TREATED_BY_EMT,NUM_RESIDENT_BUILDING_AFFCTD,NUM_BUSINESS_BUILDING_AFFCTD,ACCIDENT_PSIG,GAS_FLOW_IN_PIPE_IN_MCF
0,5/18/2022,2022,20220056,"ENABLE GAS TRANSMISSION, LLC",OK,35.296780,-93.429780,NATURAL GAS,100.0,523.0,0.0,OTHER INCIDENT CAUSE,UNKNOWN,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,UNDERGROUND,"ONSHORE PIPELINE, INCLUDING VALVE SITES",UNKNOWN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CARBON STEEL,LEAK,PINHOLE,NaN,0.0,659.0,3447.0,466421.0,1000.0,0.0,NaN,6.59,0.0,0.0,0.0,0.0,700.0,6000.0
1,5/13/2022,2022,20220055,"ARCHAEA ENERGY OPERATING, LLC",MI,35.319820,-97.360050,NATURAL GAS,961.0,0.0,0.0,INCORRECT OPERATION,OTHER INCORRECT OPERATION,NO,NaN,NaN,NaN,NaN,NaN,0,YES,1.0,0.0,0.0,0.0,0.0,1,961.0,NO,ABOVEGROUND,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2005,PIPE,PIPE BODY,12.0,0.25,24000,X-60,FURNACE BUTT WELDED,RED MAN PIPE AND SUPPLY COMPANY,EPOXY,NaN,NO,CARBON STEEL,OTHER,NaN,GAS RELEASED AND IGNITED DURING ROUTINE PIGGIN...,0.0,8072.0,0.0,80000.0,0.0,0.0,NaN,8.40,0.0,1.0,0.0,0.0,160.0,288.0
2,5/09/2022,2022,20220054,TRANSCONTINENTAL GAS PIPE LINE COMPANY,OK,32.204084,-87.949010,NATURAL GAS,53185.0,3424.0,0.0,OTHER INCIDENT CAUSE,UNKNOWN,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,UNDERGROUND,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1959,PIPE,PIPE SEAM,30.0,0.50,52000,API 5L,DSAW,UNKNOWN,COAL TAR,NaN,UNKNOWN,CARBON STEEL,RUPTURE,NaN,NaN,13000.0,239333.0,15408.0,0.0,0.0,0.0,TOTAL COSTS TBD.,4.50,0.0,0.0,1.0,0.0,779.0,40000.0
3,5/09/2022,2022,20220053,"ENABLE GAS TRANSMISSION, LLC",OK,35.691940,-100.227520,NATURAL GAS,6975.0,0.0,0.0,EQUIPMENT FAILURE,MALFUNCTION OF CONTROL/RELIEF EQUIPMENT,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,ABOVEGROUND,ONSHORE REGULATOR/METERING STATION EQUIPMENT A...,1978,VALVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CARBON STEEL,OTHER,NaN,PREMATURE GAS RELEASE,0.0,34178.0,0.0,1000.0,1000.0,0.0,NaN,4.90,0.0,0.0,0.0,0.0,967.0,2000.0
4,4/27/2022,2022,20220049,"GULF SOUTH PIPELINE COMPANY, LLC",TX,32.028480,-90.142318,NATURAL GAS,3394.0,0.0,0.0,EQUIPMENT FAILURE,OTHER EQUIPMENT FAILURE,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,ABOVEGROUND,ONSHORE COMPRESSOR STATION EQUIPMENT AND PIPING,2007,"ESD SYSTEM, BUT EXCLUDING PRODUCT DRAIN LINES ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CARBON STEEL,OTHER,NaN,COMPRESSOR STATION ESD,0.0,18124.0,0.0,651.0,630.0,0.0,NaN,5.34,0.0,0.0,0.0,0.0,1424.0,2300.0


### Make a plotly bar plot with incident count, year and state

In [6]:
grouped_Year_State = HGL_trans_acc.groupby(['IYEAR','OPERATOR_STATE_ABBREVIATION']).count().sort_values(by = 'REPORT_NUMBER',  ascending=False).reset_index()
grouped_Year_State.head()

# The column report number (1 on 1 relationship) can be our incident count so:

grouped_Year_State.rename(columns = {'IYEAR':'Incident Year','OPERATOR_STATE_ABBREVIATION':'Incident State', 'REPORT_NUMBER':'No of Incidents'}, inplace=True)
grouped_Year_State.sample(3)

,Incident Year,Incident State,REPORT_RECEIVED_DATE,No of Incidents,NAME,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,ACCOMPANYING_LIQUID,CAUSE,CAUSE_DETAILS,FATALITY_IND,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,FATAL,INJURY_IND,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,INJURE,GAS_CONSUMED_BY_FIRE_IN_MCF,EXPLODE_IND,INCIDENT_AREA_TYPE,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,ITEM_INVOLVED,PIPE_TYPE,PIPE_DIAMETER,PIPE_WALL_THICKNESS,PIPE_SMYS,PIPE_SPECIFICATION,PIPE_SEAM_TYPE,PIPE_MANUFACTURER,PIPE_COATING_TYPE,PIPE_COATING_DETAILS,COATING_APPLIED_IND,MATERIAL_INVOLVED,RELEASE_TYPE,LEAK_TYPE,RELEASE_TYPE_DETAILS,EST_COST_OPER_PAID,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,EST_COST_OTHER_DETAILS,GAS_COST_IN_MCF,NUM_PERSONS_HOSP_NOT_OVNGHT,NUM_INJURED_TREATED_BY_EMT,NUM_RESIDENT_BUILDING_AFFCTD,NUM_BUSINESS_BUILDING_AFFCTD,ACCIDENT_PSIG,GAS_FLOW_IN_PIPE_IN_MCF
122,2012,CO,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,0
79,2016,CO,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,2,2,0,0,0,0,0,2,0,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,2,2,1,1,2,2,2,2,2,2,0,0,0,0,0,0,2,0
66,2019,NE,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,3,3,0,0,0,0,0,3,0,3,3,3,3,3,2,2,2,2,2,2,2,2,0,0,3,3,1,0,3,3,3,3,3,3,0,0,0,0,0,0,3,0


In [7]:
#plotting using plotly:

# #trying with plotly express
fig = px.bar(data_frame=grouped_Year_State, y = 'Incident Year', x = 'No of Incidents',
            color = 'Incident State', orientation='h', 
            color_discrete_sequence= px.colors.sequential.Electric,
            title='Number of Gas Transmission Line Incidents in Past 10 Years by State') #remove latitude form pipe)

fig.update_layout(showlegend=False)
fig.show()


Good reference for the colour pallets. Only the sequential one works here. 
- dir(px.colors.qualitative)
- dir(px.colors.diverging)
- dir(px.colors.sequential)

## Having a look at the Injuries, Fatalities and Cuases of the transmission incidents

In [9]:
print('The fatality count is:')
print(HGL_trans_acc['FATAL'].value_counts())
print("\n")
print('The injury count is:')
print(HGL_trans_acc['INJURE'].value_counts())

The fatality count is:
0    1495
1      11
2       4
4       1
8       1
Name: FATAL, dtype: int64


The injury count is:
0     1479
1       22
2        5
4        2
6        1
13       1
51       1
7        1
Name: INJURE, dtype: int64


In [10]:
Acc_Cause = HGL_trans_acc.groupby(['CAUSE','CAUSE_DETAILS']).count().reset_index()
Acc_Cause.rename(columns = {'REPORT_NUMBER':'Number of Incidents','CAUSE':'Accident Cause', 'CAUSE_DETAILS':'Details'}, inplace=True) #each incident has a report number so its count is incident count

Acc_Cause['Accident Cause'] = Acc_Cause['Accident Cause'].str.lower().str.title()
Acc_Cause['Details'] = Acc_Cause['Details'].str.lower().str.capitalize()

Acc_Cause.head(3)


,Accident Cause,Details,REPORT_RECEIVED_DATE,IYEAR,Number of Incidents,NAME,OPERATOR_STATE_ABBREVIATION,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,ACCOMPANYING_LIQUID,FATALITY_IND,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,FATAL,INJURY_IND,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,INJURE,GAS_CONSUMED_BY_FIRE_IN_MCF,EXPLODE_IND,INCIDENT_AREA_TYPE,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,ITEM_INVOLVED,PIPE_TYPE,PIPE_DIAMETER,PIPE_WALL_THICKNESS,PIPE_SMYS,PIPE_SPECIFICATION,PIPE_SEAM_TYPE,PIPE_MANUFACTURER,PIPE_COATING_TYPE,PIPE_COATING_DETAILS,COATING_APPLIED_IND,MATERIAL_INVOLVED,RELEASE_TYPE,LEAK_TYPE,RELEASE_TYPE_DETAILS,EST_COST_OPER_PAID,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,EST_COST_OTHER_DETAILS,GAS_COST_IN_MCF,NUM_PERSONS_HOSP_NOT_OVNGHT,NUM_INJURED_TREATED_BY_EMT,NUM_RESIDENT_BUILDING_AFFCTD,NUM_BUSINESS_BUILDING_AFFCTD,ACCIDENT_PSIG,GAS_FLOW_IN_PIPE_IN_MCF
0,Corrosion Failure,External corrosion,104,104,104,104,104,104,104,102,104,97,94,104,0,0,0,0,0,104,104,3,3,3,3,3,104,3,81,87,104,102,104,97,98,98,97,98,98,97,98,16,25,104,104,69,0,104,104,103,104,104,103,23,26,26,26,26,26,104,26
1,Corrosion Failure,Internal corrosion,183,183,183,183,183,183,183,181,183,168,164,183,0,0,0,0,0,183,183,0,0,0,0,0,183,3,155,68,183,179,183,163,171,171,171,171,171,169,171,88,29,183,183,154,0,183,183,183,183,183,182,33,31,31,31,31,31,183,31
2,Equipment Failure,Compressor or compressor-related equipment,31,31,31,31,31,31,31,29,31,31,30,31,0,0,0,0,0,31,31,1,1,1,1,1,31,0,26,31,31,28,31,0,0,0,0,0,0,0,0,0,0,31,31,9,22,31,31,31,31,31,31,8,5,5,5,5,5,31,5


In [11]:

# #trying with plotly express
fig = px.bar(data_frame=Acc_Cause, y = 'Accident Cause', x = 'Number of Incidents', color = 'Details',
            color_discrete_sequence= px.colors.sequential.ice, orientation='h', 
            title='Causes of Gas Transmission Line Incidents in Past 10 Years',hover_data={'Accident Cause':False}) #remove latitude form pipe)
fig.update_layout(showlegend=False)
fig.show()


# Incidents in the Gas Distribution Network 

In [12]:
# Working on Gas Distribution Accidents now:
#changed encoding by saving as csv file again, tab delimited
Gas_Dist_Accidents = pd.read_csv('data/incident_gas_distribution_jan2010_present.csv',low_memory=False) 
Gas_Dist_Accidents.sample(2)
Gas_Dist_Accidents.shape


(1314, 466)

In [13]:
req_clmns =  []
with open('data/Gas_Dist_required_columns.txt') as f: #importing the list of required columns form the text file
    lines = f.readlines()
f.close()
for l in lines:
    req_clmns.append(l.strip()) #getting rid of the "\n" escape character
Gas_Dist_Acc = Gas_Dist_Accidents[req_clmns]

Gas_Dist_Acc.shape
Gas_Dist_Acc.sample(3)

,REPORT_RECEIVED_DATE,IYEAR,NAME,LOCATION_STREET_ADDRESS,LOCATION_CITY_NAME,LOCATION_COUNTY_NAME,LOCATION_STATE_ABBREVIATION,LOCATION_POSTAL_CODE,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,FATALITY_IND,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,FATAL,INJURY_IND,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,INJURE,IGNITE_IND,IGNITE_DATETIME,HOW_EXTINGUISHED,GAS_CONSUMED_BY_FIRE_IN_MCF,EXPLODE_IND,NUM_PUB_EVACUATED,INCIDENT_AREA_TYPE,INCIDENT_AREA_SUBTYPE,INCIDENT_AREA_DETAILS,MATERIAL_INVOLVED,MATERIAL_DETAILS,RELEASE_TYPE,PUNCTURE_AXIAL,PUNCTURE_CIRCUM,CLASS_LOCATION_TYPE,EST_COST_OPER_PAID,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,GAS_COST_IN_MCF,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,CAUSE_DETAILS,CAUSE
215,1/03/2020,2019,PHILADELPHIA GAS WORKS,1435 S. 8TH STREET,PHILADELPHIA,PHILADELPHIA,PA,19147,39.930770,-75.157868,NATURAL GAS,10.0,NaN,YES,0.0,0.0,0.0,0.0,2.0,2,NO,NaN,NaN,NaN,NaN,NaN,0,YES,NaN,NaN,NaN,YES,60.0,UNDERGROUND,UNDER PAVEMENT,NaN,CAST/WROUGHT IRON,NaN,RUPTURE,NaN,NaN,CLASS 4 LOCATION,1375900.0,1000.0,40000.0,0.0,NaN,20.0,NaN,OTHER OUTSIDE FORCE DAMAGE,OTHER OUTSIDE FORCE DAMAGE
1060,3/13/2012,2012,METROPOLITAN UTILITIES DISTRICT,4213 S 37TH STREET,OMAHA,DOUGLAS,NE,68107,41.218051,-95.968372,NATURAL GAS,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,YES,NaN,NaN,NaN,YES,0.0,ABOVEGROUND,INSIDE A BUILDING,NaN,STEEL,NaN,LEAK,NaN,NaN,CLASS 1 LOCATION,50000.0,2000.0,0.0,0.0,NaN,1000.0,NaN,INTENTIONAL DAMAGE,OTHER OUTSIDE FORCE DAMAGE
758,3/26/2015,2015,PACIFIC GAS & ELECTRIC CO,ELK-GROVE FLORIN RD AND GERBER RD,SACRAMENTO,SACRAMENTO,CA,95829,38.481793,-121.367310,NATURAL GAS,293.0,NaN,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NaN,NaN,0,NO,NaN,NaN,NaN,NO,0.0,UNDERGROUND,UNDER SOIL,NaN,PLASTIC,NaN,MECHANICAL PUNCTURE,1.0,1.0,CLASS 3 LOCATION,0.0,163600.0,0.0,0.0,NaN,1260.0,NaN,EXCAVATION DAMAGE BY THIRD PARTY,EXCAVATION DAMAGE


There is an outlier in the longitude column (large negative number), so we find it and drop that whole row so we can do the plotting 

In [14]:
drop_row_index = Gas_Dist_Acc['LOCATION_LONGITUDE'].idxmin() #getting the row index for the minimum oulier 
Gas_Dist_Acc.drop(Gas_Dist_Acc.index[drop_row_index], inplace = True)
Gas_Dist_Acc.shape #So we dropped one row

C:\Users\Ramin\AppData\Local\Temp\ipykernel_1632\970168688.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(1313, 51)

Lets try plotting the incident locations again

In [15]:
Gas_Dist_Acc["IYEAR"].info()

<class 'pandas.core.series.Series'>
Int64Index: 1313 entries, 0 to 1313
Series name: IYEAR
Non-Null Count  Dtype
--------------  -----
1313 non-null   int64
dtypes: int64(1)
memory usage: 20.5 KB


In [16]:
# I have discovered some outliers that fall outside the continental USA, so I will explude those: 
Gas_Dist_Acc = Gas_Dist_Acc[(-140<Gas_Dist_Acc['LOCATION_LONGITUDE']) & (Gas_Dist_Acc['LOCATION_LONGITUDE']<-50)]
Gas_Dist_Acc = Gas_Dist_Acc[Gas_Dist_Acc['LOCATION_LATITUDE']<50]

#Formatting  
Gas_Dist_Acc['hover_text'] = Gas_Dist_Acc["LOCATION_CITY_NAME"].str.title()+", "+ Gas_Dist_Acc["LOCATION_STATE_ABBREVIATION"]+ ", "+"Cause: "+Gas_Dist_Acc['CAUSE'].str.lower().str.capitalize()+", " + Gas_Dist_Acc["IYEAR"].astype(str)

fig = go.Figure(data = go.Scattergeo(
                    lat=Gas_Dist_Acc.LOCATION_LATITUDE, lon=Gas_Dist_Acc.LOCATION_LONGITUDE,
                    text=Gas_Dist_Acc['hover_text'], 
                    mode = 'markers',
                   marker = dict(
                    size = 8,
                    opacity = 0.8,
                    reversescale = True,
                    autocolorscale = False,
                    symbol = 'triangle-up',
                    line = dict(
                        width=1,
                        color='rgba(102, 102, 102)'
                    ),
                    colorscale = 'greens',
                    cmin = 2010,
                    color = Gas_Dist_Acc.IYEAR,
                    cmax = Gas_Dist_Acc.IYEAR.max(),
                    colorbar_title="Incident Year")))

fig.update_layout(
        title = 'Gas distribution line incidents in the last 10 years',
        geo_scope='usa'
    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [17]:
#Using Scatter Mapbox for a different type of location plot

fig = px.scatter_mapbox(Gas_Dist_Acc, lat="LOCATION_LATITUDE", lon="LOCATION_LONGITUDE", hover_name="LOCATION_CITY_NAME",
                    hover_data=["CAUSE_DETAILS", "IYEAR"],
                    color_discrete_sequence=["red"], zoom=3, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [43]:
grouped_Year_State = Gas_Dist_Acc.groupby(['IYEAR','LOCATION_STATE_ABBREVIATION']).count().sort_values(by = 'NAME',  ascending=False).reset_index()

#creating a data frame as the output of the groupby. The product of the groupby is a series, 
# so you do a to_frame to get the df and then reset the index to get a proper index
# The column NAME (1 on 1 relationship) can be our incident count so:

grouped_Year_State.rename(columns = {'IYEAR':'Incident Year','LOCATION_STATE_ABBREVIATION':'Incident State', 'NAME':'No of Incidents'}, inplace=True)
#Sorting based on year
grouped_Year_State=grouped_Year_State.sort_values("Incident Year") # Make sure you sort the time horizon column in ascending order

grouped_Year_State.sample(3)



,Incident Year,Incident State,REPORT_RECEIVED_DATE,No of Incidents,LOCATION_STREET_ADDRESS,LOCATION_CITY_NAME,LOCATION_COUNTY_NAME,LOCATION_POSTAL_CODE,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,FATALITY_IND,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,FATAL,INJURY_IND,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,INJURE,IGNITE_IND,IGNITE_DATETIME,HOW_EXTINGUISHED,GAS_CONSUMED_BY_FIRE_IN_MCF,EXPLODE_IND,NUM_PUB_EVACUATED,INCIDENT_AREA_TYPE,INCIDENT_AREA_SUBTYPE,INCIDENT_AREA_DETAILS,MATERIAL_INVOLVED,MATERIAL_DETAILS,RELEASE_TYPE,PUNCTURE_AXIAL,PUNCTURE_CIRCUM,CLASS_LOCATION_TYPE,EST_COST_OPER_PAID,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,GAS_COST_IN_MCF,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,CAUSE_DETAILS,CAUSE,hover_text
392,2017,MS,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,0,1,0,1,1,1
351,2015,MD,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,1,1,1
227,2014,KS,2,2,2,2,2,2,2,2,2,2,0,2,0,0,0,0,0,2,2,1,1,1,1,1,2,2,0,0,0,2,2,2,2,0,2,0,2,0,0,2,2,2,2,2,0,2,0,2,2,2


In [40]:
grouped_Year_State['Incident Year'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 399 entries, 0 to 398
Series name: Incident Year
Non-Null Count  Dtype
--------------  -----
399 non-null    int64
dtypes: int64(1)
memory usage: 3.2 KB


In [44]:
#plotting using plotly:

# #trying with plotly express
fig = px.bar(data_frame=grouped_Year_State, y = 'Incident Year', x = 'No of Incidents',
            color = 'Incident State', orientation='h', 
            color_discrete_sequence= px.colors.sequential.Plotly3,
            title='Number of Gas Distribution Incidents in Past 10 Years by State') #remove latitude form pipe)

fig.update_layout(showlegend=False)
fig.show()


The above is not very telling and difficult to interpret even with the hover text. So we do a Chorlopleth Map for every year. 

### Animated Chorlopleth Map to understand the gas distribution incident data better.
https://towardsdatascience.com/simplest-way-of-creating-a-choropleth-map-by-u-s-states-in-python-f359ada7735e

In [153]:
import plotly.express as px
fig = px.choropleth(grouped_Year_State,
                    locations='Incident State', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='No of Incidents',
                    color_continuous_scale="sunset", 
                    animation_frame='Incident Year' #make sure 'Incident Year' is string type and sorted in ascending order
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

So in the geopandas states data frame the state abbreviation is 'STUSPS'. In the acc_state (accident by state) dataframe we have made, the state abbreviation is: Satates, 
we can merge these two and replace any NaN with zero for our heatmap.
I got the instructions from here: https://medium.com/@m_vemuri/create-a-geographic-heat-map-of-the-city-of-toronto-in-python-cd2ae0f8be55

## Having a look at the Injuries, Fatalities and Cuases of the transmission incidents

In [78]:
print('The fatality count is:')
print(Gas_Dist_Acc['FATAL'].value_counts())
print("\n")
print('The injury count is:')
print(Gas_Dist_Acc['INJURE'].value_counts())

The fatality count is:
0    1222
1      64
2      10
3       2
7       1
8       1
5       1
Name: FATAL, dtype: int64


The injury count is:
0     1027
1      182
2       48
3       17
4       11
6        6
10       2
5        2
25       1
33       1
48       1
8        1
17       1
7        1
Name: INJURE, dtype: int64


In [97]:
Fatalities.shape

(311, 52)

The injury/fatality rate happens to be larger than the Gas Transmission lines injury/fatality, so this warrants a closer look. We combine the fatalities and the injuries into a new column "Casualties".

In [132]:
Casualties_by_State.sample(3)

,State,Cause,IYEAR,LOCATION_LATITUDE,LOCATION_LONGITUDE,UNINTENTIONAL_RELEASE,INTENTIONAL_RELEASE,NUM_EMP_FATALITIES,NUM_CONTR_FATALITIES,NUM_ER_FATALITIES,NUM_WORKER_FATALITIES,NUM_GP_FATALITIES,Number_Fatalities,NUM_EMP_INJURIES,NUM_CONTR_INJURIES,NUM_ER_INJURIES,NUM_WORKER_INJURIES,NUM_GP_INJURIES,Number_Injuried,GAS_CONSUMED_BY_FIRE_IN_MCF,NUM_PUB_EVACUATED,PUNCTURE_AXIAL,PUNCTURE_CIRCUM,EST_COST_OPER_PAID,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_OTHER,GAS_COST_IN_MCF,EST_COST_UNINTENTIONAL_RELEASE,EST_COST_INTENTIONAL_RELEASE,Casualties
85,NJ,Material failure of pipe or weld,4035,80.426625,-148.331063,11802.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,1.0,3,0.0,2.0,0.0,0.0,374000.0,449363.0,297011.0,131921.0,0.0,18203.0,0.0,3
104,OK,Incorrect operation,2018,36.313150,-96.463191,67.5,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,2,0.0,50.0,0.0,0.0,0.0,1000.0,500.0,0.0,0.0,270.0,0.0,2
47,KY,Natural force damage,2013,39.091901,-84.462267,35.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.0,0.0,0.0,0.0,1.0,1,0.0,0.0,0.0,0.0,89200.0,45.0,1500.0,0.0,0.0,142.0,0.0,2


In [130]:

Casualties = Gas_Dist_Acc[(Gas_Dist_Acc['FATALITY_IND']=='YES')|(Gas_Dist_Acc['INJURY_IND']=='YES')]
Casualties_by_State = Casualties.groupby(['LOCATION_STATE_ABBREVIATION', 'CAUSE']).sum()
Casualties_by_State= Casualties_by_State.reset_index()
Casualties_by_State.rename(columns = {'LOCATION_STATE_ABBREVIATION':'State',
                                    'FATAL':'Number_Fatalities',
                                    'CAUSE': 'Cause',
                                    'INJURE':'Number_Injuried'}, inplace=True)


Casualties_by_State['Cause'] = Casualties_by_State['Cause'].str.lower().str.capitalize()
Casualties_by_State['Casualties'] = Casualties_by_State['Number_Fatalities']+Casualties_by_State['Number_Injuried']

fig = px.bar(data_frame=Casualties_by_State, y = 'Cause', x = 'Casualties', color = 'State', #hover_data='CAUSE_DETAILS',
            color_discrete_sequence= px.colors.sequential.ice, orientation='h', 
            title='Casualties of Gas Distribution Line Incidents in Past 10 Years')
fig.update_layout(showlegend=False)
fig.show()



Using a map to show the worst States in terms of the total casualties in the last 10 years. 

In [148]:
Casualties_by_State =Casualties_by_State.groupby(['State']).sum().reset_index() #grouping based on States only for the heatmap

In [154]:
#Plotting the heatmap
fig = px.choropleth(Casualties_by_State,
                    locations='State', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Casualties',
                    color_continuous_scale="speed", 
                    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()